#**CP631 - Assignment 3 - PySpark on Titanic Trouble**

**Name:** Ma Luo
**Email:** Luox1180@mylaurier.ca

**Please run Part A code in the first session before we start here**

##**Before Runing**

In [ ]:
##Install Apache Spark 3.0.0
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

##Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

#Init Find Spark
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [46]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Part I - Load Training Set**

In [84]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

import pyspark
conf = pyspark.SparkConf().setAppName("TitanicNotebook") #Defines the Spark App firstly
try:
    sc = pyspark.SparkContext(conf=conf)
except:
    print("Warning : a SparkContext already exists.")

spark = SparkSession \
    .builder \
    .appName("TitanicNotebook") \
    .getOrCreate()    

test_path = "/content/drive/MyDrive/Colab Notebooks/CP631/Titanic/test.csv"
train_path = "/content/drive/MyDrive/Colab Notebooks/CP631/Titanic/train.csv"

titanic_df = spark.read.csv(train_rdd,header = 'True',inferSchema='True')


##**Part II - Play Training Set**

In [89]:
titanic_df.take(3)
display(titanic_df)

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [88]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [91]:
passengers_count = titanic_df.count()
print(passengers_count)

891


In [92]:
titanic_df.show(5)


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [93]:
titanic_df.describe().show()


+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+------------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|              Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+------------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|               891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738|32.20

In [95]:
titanic_df.select("Survived","Pclass","Embarked").show()


+--------+------+--------+
|Survived|Pclass|Embarked|
+--------+------+--------+
|       0|     3|       S|
|       1|     1|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       Q|
|       0|     1|       S|
|       0|     3|       S|
|       1|     3|       S|
|       1|     2|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       S|
|       0|     3|       S|
|       1|     2|       S|
|       0|     3|       Q|
|       1|     2|       S|
|       0|     3|       S|
|       1|     3|       C|
+--------+------+--------+
only showing top 20 rows



##**Part III - Explore more**

### **Survival STATS**

In [96]:
titanic_df.groupBy("Survived").count().show()


+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [97]:
gropuBy_output = titanic_df.groupBy("Survived").count()


In [99]:
display(gropuBy_output)


DataFrame[Survived: int, count: bigint]

In [101]:
titanic_df.groupBy("Sex","Survived").count().show()


+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



### **Checking Null Values**

In [102]:
# This function use to print feature with null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

#### **NULL Sum**

In [103]:
null_columns_count_list = null_value_count(titanic_df)


In [104]:
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()


+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                   Age|              177|
|                 Cabin|              687|
|              Embarked|                2|
+----------------------+-----------------+



### **Check Initals By Age**

In [ ]:
titanic_df.select("Name").show()


In [ ]:
titanic_df = titanic_df.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))
titanic_df.show()


In [110]:
titanic_df.select("Initial").distinct().show()


+--------+
| Initial|
+--------+
|     Don|
|    Miss|
|Countess|
|     Col|
|     Rev|
|    Lady|
|  Master|
|     Mme|
|    Capt|
|      Mr|
|      Dr|
|     Mrs|
|     Sir|
|Jonkheer|
|    Mlle|
|   Major|
|      Ms|
+--------+



In [111]:
#Fix some typos here
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [112]:
titanic_df.select("Initial").distinct().show()


+-------+
|Initial|
+-------+
|   Miss|
|  Other|
| Master|
|     Mr|
|    Mrs|
+-------+



In [113]:
titanic_df.groupby('Initial').avg('Age').collect()


[Row(Initial='Miss', avg(Age)=21.86),
 Row(Initial='Other', avg(Age)=45.888888888888886),
 Row(Initial='Master', avg(Age)=4.574166666666667),
 Row(Initial='Mr', avg(Age)=32.73960880195599),
 Row(Initial='Mrs', avg(Age)=35.981818181818184)]

In [114]:
#igore the nulls
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Miss") & (titanic_df["Age"].isNull()), 22).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Other") & (titanic_df["Age"].isNull()), 46).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Master") & (titanic_df["Age"].isNull()), 5).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Mr") & (titanic_df["Age"].isNull()), 33).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Mrs") & (titanic_df["Age"].isNull()), 36).otherwise(titanic_df["Age"]))

In [116]:
titanic_df.groupby('Initial').avg('Age').collect()


[Row(Initial='Miss', avg(Age)=21.887096774193548),
 Row(Initial='Other', avg(Age)=45.888888888888886),
 Row(Initial='Master', avg(Age)=4.616750000000001),
 Row(Initial='Mr', avg(Age)=32.79867674858223),
 Row(Initial='Mrs', avg(Age)=35.98425196850393)]

### **Drop columns which are not required**

In [117]:
titanic_df = titanic_df.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","Initial")


In [ ]:
titanic_df.show()


### **Bring features to vectors and prepare for modelling**

In [120]:
feature = VectorAssembler(inputCols=titanic_df.columns[1:],outputCol="features")
feature_vector= feature.transform(titanic_df)
feature_vector.show()

+--------+------+----+-----+-----+-------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|            features|
+--------+------+----+-----+-----+-------+--------------------+
|       0|     3|22.0|    1|    0|   7.25|[3.0,22.0,1.0,0.0...|
|       1|     1|38.0|    1|    0|71.2833|[1.0,38.0,1.0,0.0...|
|       1|     3|26.0|    0|    0|  7.925|[3.0,26.0,0.0,0.0...|
|       1|     1|35.0|    1|    0|   53.1|[1.0,35.0,1.0,0.0...|
|       0|     3|35.0|    0|    0|   8.05|[3.0,35.0,0.0,0.0...|
|       0|     3|33.0|    0|    0| 8.4583|[3.0,33.0,0.0,0.0...|
|       0|     1|54.0|    0|    0|51.8625|[1.0,54.0,0.0,0.0...|
|       0|     3| 2.0|    3|    1| 21.075|[3.0,2.0,3.0,1.0,...|
|       1|     3|27.0|    0|    2|11.1333|[3.0,27.0,0.0,2.0...|
|       1|     2|14.0|    1|    0|30.0708|[2.0,14.0,1.0,0.0...|
|       1|     3| 4.0|    1|    1|   16.7|[3.0,4.0,1.0,1.0,...|
|       1|     1|58.0|    0|    0|  26.55|[1.0,58.0,0.0,0.0...|
|       0|     3|20.0|    0|    0|   8.0

In [121]:
(trainingData, testData) = feature_vector.randomSplit([0.8, 0.2],seed = 11)


### **Modelling - LogisticRegression**

In [122]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Survived", featuresCol="features")
#Training algo
lrModel = lr.fit(trainingData)
lr_prediction = lrModel.transform(testData)
lr_prediction.select("prediction", "Survived", "features").show()
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[1.0,24.0,0.0,1.0...|
|       1.0|       0|[1.0,27.0,0.0,2.0...|
|       1.0|       0|[1.0,33.0,0.0,0.0...|
|       1.0|       0|[1.0,33.0,0.0,0.0...|
|       1.0|       0|[1.0,33.0,0.0,0.0...|
|       1.0|       0|[1.0,33.0,0.0,0.0...|
|       1.0|       0|[1.0,37.0,0.0,1.0...|
|       1.0|       0|[1.0,40.0,0.0,0.0...|
|       0.0|       0|[1.0,46.0,1.0,0.0...|
|       0.0|       0|[1.0,50.0,0.0,0.0...|
|       1.0|       0|[1.0,52.0,1.0,1.0...|
|       0.0|       0|[1.0,62.0,0.0,0.0...|
|       0.0|       0|[1.0,65.0,0.0,1.0...|
|       1.0|       0|[2.0,21.0,0.0,0.0...|
|       0.0|       0|[2.0,24.0,0.0,0.0...|
|       0.0|       0|[2.0,31.0,0.0,0.0...|
|       0.0|       0|(5,[0,1],[2.0,33.0])|
|       0.0|       0|[2.0,34.0,1.0,0.0...|
|       0.0|       0|[2.0,36.0,0.0,0.0...|
|       0.0|       0|[2.0,54.0,0.0,0.0...|
+----------